# Set up for ML

We'd like to try to replicate the approach these guys took

https://github.com/jorgemarpa/PELS-VAE / https://github.com/jorgemarpa/PPDAE 
https://arxiv.org/abs/2005.07773 
https://ui.adsabs.harvard.edu/abs/2022ApJ...939...73T/abstract

to speed up RT calculations.

A few options exist:

 1. Fully predict SED (in each aperture?) based on model paramters
 2. Predict temperature map as a function of r, z given model parameters
 3. Predict temperature _and_ density map as a function of r, z given model parameters
 
I'm going to try (2) first

In [1]:
cd '/blue/adamginsburg/richardson.t/research/flux'

/blue/adamginsburg/richardson.t/research/flux


In [2]:
import numpy as np
from user_functions import metric_distance
from astropy.table import Table
from sedfitter.sed import SEDCube
from scipy.spatial import KDTree
from tqdm import tqdm

def get_mass(table,row,ap,key):
    return np.nanmax(table[key][row,:ap])

In [3]:
from sedfitter.sed import SEDCube
from astropy.table import Table
from astropy import units as u

In [4]:
%matplotlib inline
import pylab as pl
pl.rcParams['font.size'] = 16
pl.style.use('dark_background')

In [5]:
from hyperion import model

In [6]:
geometry='s-p-smi'
incs=1
ap=10
wav=24
norm='none'
neighbors=10

In [7]:
pars = Table.read(f'robitaille_models-1.2/{geometry}/augmented_parameters.fits')
pars.add_column(range(0,len(pars)),name='Index')
reduced_pars = pars[::incs]

all_T = reduced_pars['star.temperature']
all_L = reduced_pars['Model Luminosity']

env_masses = []
for row in range(len(reduced_pars)):
    env_masses.append(get_mass(pars,row,ap,'Sphere Masses'))
env_masses = np.array(env_masses)

if (norm == 'none') or (norm == 'unnormed_metric'):
    fulldat = np.array((np.log10(all_T),
                        np.log10(all_L),
                        np.log10(env_masses)))
else:
    t_norm, l_norm, m_norm = log_range(all_T), log_range(all_L), log_range(env_masses)
    fulldat = np.array((np.log10(all_T)/t_norm, np.log10(all_L)/l_norm, np.log10(env_masses)/m_norm))

valid_models = np.all(np.isfinite(fulldat), axis=0)
fulldat = fulldat[:,valid_models]
kt = KDTree(fulldat.T)
valid_pars = reduced_pars[valid_models]

if 'metric' in norm:
    dists,locs = metric_distance(fulldat,fulldat,neighbors+1,norm)
else:
    dists,locs = kt.query(fulldat.T,neighbors+1)

row_indices = []
near_indices = []
for row in range(locs.shape[0]):
    model_index = locs[row,0]
    pars_index = valid_pars['Index'][model_index]
    row_indices.append([value for value in range(pars_index,pars_index+incs)])

    indices = []
    for idx in locs[row,1:]: #leave the point being checked out
        pars_index = valid_pars['Index'][idx]
        indices.extend(range(pars_index,pars_index+incs))
    near_indices.append(indices)

names = [name[:8] for name in valid_pars['MODEL_NAME']]
seds = SEDCube.read(f'robitaille_models-1.2/{geometry}/flux.fits')
row_fluxes = dict(zip(names, seds.val[np.array(row_indices),ap,wav]))
near_fluxes = dict(zip(names, seds.val[np.array(near_indices),ap,wav]))
#row_fluxes = {}
#near_fluxes = {}
# for i in tqdm(range(len(names))):
#     row_fluxes.update({names[i]:seds.val[row_indices[i],ap,wav]})
#     near_fluxes.update({names[i]:seds.val[near_indices[i],ap,wav]})
# fluxtable = Table([[key for key in row_fluxes],[value for value in row_fluxes.values()],[value for value in near_fluxes.values()]],names=['Model Name','Model Fluxes','Neighbor Fluxes'])

/scratch/local/5800391/ipykernel_47726/1053669639.py:11: UserWarning: Warning: converting a masked element to nan.
  env_masses = np.array(env_masses)


In [8]:
len(near_indices), len(near_indices[0])

(8137, 10)

In [9]:
seds.val[np.array(near_indices),ap,wav]

<Quantity [[          nan,           nan,           nan, ...,
                      nan,           nan,           nan],
           [          nan,           nan,           nan, ...,
                      nan,           nan,           nan],
           [1.3843965e-02, 1.0422094e-02, 1.9184722e-02, ...,
            1.6173294e-02, 4.4618002e-03, 1.0288118e-02],
           ...,
           [2.5378033e+01, 7.0624146e+01, 6.3063274e+01, ...,
            8.0082748e+01, 5.8148258e+01, 8.1505440e+01],
           [5.1767619e+05, 5.6171038e+05, 3.0210888e+05, ...,
            3.1928319e+05, 1.0155259e+06, 2.1777956e+05],
           [4.0587350e+05, 4.4125681e+05, 3.5428547e+05, ...,
                      nan, 3.3222103e+05,           nan]] mJy>

In [10]:
seds.val[np.array(near_indices),ap,wav].shape

(8137, 10)

In [11]:
near_fluxes = dict(zip(names, seds.val[np.array(near_indices)[:,:,None],
                                       ap,
                                       np.arange(200)[None,None,:]]))

In [12]:
near_fluxes['01NcC57M'].shape

(10, 200)

In [13]:
seds.val.shape

(10000, 20, 200)

# Load 2D arrays

First thing to try: Predict temperature based on model parameters

In [14]:
mod = model.ModelOutput('/blue/adamginsburg/richardson.t/research/flux/grids-1.1/spubhmi/output/0h/0hO4CPtf.rtout')

In [15]:
qtys = mod.get_quantities()

INFO: No density present in output, reading initial density [hyperion.model.model_output]


In [16]:
qtys['temperature'].shape

(1, 300, 400)

## Make a grid of 10,000 images

for now we'll limit to 1 geometry, a simple powerlaw

In [17]:
from tqdm.auto import tqdm

In [21]:
rootdir = '/orange/adamginsburg/robitaille_models/ML_PPDAE/'

In [22]:
gridpath = f'/blue/adamginsburg/richardson.t/research/flux/grids-1.1/{geometry}/output/'

In [ ]:
pars = Table.read(f'robitaille_models-1.2/{geometry}/augmented_parameters.fits')

In [20]:
npars = len(pars)

In [25]:
mn = pars['MODEL_NAME'][0]
tem = model.ModelOutput(f'{gridpath}/{mn[:2]}/{mn[:-3]}.rtout').get_quantities()['temperature'][0].array
tem.shape

INFO: No density present in output, reading initial density [hyperion.model.model_output]


(1, 1, 400)

In [26]:
filesize = (npars * np.product(qtys['temperature'].shape) * 4 * u.byte).to(u.GB)
filesize

<Quantity 4.8 Gbyte>

In [27]:
arr = np.memmap(f'{rootdir}/{geometry}_temperature_grid_for_ML.npy',
                shape=(npars, *tem.shape),
                mode='r+')

In [28]:
from astropy import log
log.setLevel(0)

for ii, mn in tqdm(enumerate(pars['MODEL_NAME'])):
    arr[ii, :, :, :] = (model
                        .ModelOutput(f'{gridpath}/{mn[:2].lower()}/{mn[:-3]}.rtout')
                        .get_quantities()['temperature'][0].array
                       )
    
log.setLevel('INFO')

0it [00:00, ?it/s]

I tried this all the way through, and it broke because the AI model is explicitly 2D.  We could make a 1D version by modifying the models in ``ae_model_phy.py``, but that requires a lot more thought and a deeper understanding of the order of operations (it looks like there are already some 1D transformations happening), so I'm trying a 2D version first...

## Training setup

First, we go through all the cells manually, then we set up a script so we can do this for any grid geometry we want

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
train_idx, test_idx = train_test_split(np.arange(len(pars)), 
                                       test_size=.2, random_state=99)
train_idx.shape, test_idx.shape

((8000,), (2000,))

We need numerical parameters to fit.  We want the ones that matter.  For the 1d case, inclination does not, but for 2d cases, it does... kinda.

In [31]:
parameters_to_fit = ['star.radius', 'star.temperature', 'envelope.rho_0',
 'envelope.power', 'ambient.density', 'ambient.temperature', 'scattering',
 'inclination', 
]

In [32]:
pars_arrlike = np.array(pars[parameters_to_fit]).view(float).reshape(len(pars), len(parameters_to_fit))

In [34]:
subset_id = geometry
np.save(f'{rootdir}/param_arr_gridandfiller_{subset_id}_train_all.npy',
        pars_arrlike[train_idx]
        #meta_f1.iloc[train_idx, :-1].values.astype(np.float32),
       )
np.save(f'{rootdir}/param_arr_gridandfiller_{subset_id}_test.npy',
        pars_arrlike[test_idx]
        # meta_f1.iloc[test_idx, :-1].values.astype(np.float32))
       )

In [35]:
imgs = arr
np.save(f'{rootdir}/img_array_gridandfiller_imagenorm_{subset_id}_train_all.npy', imgs[train_idx])
np.save(f'{rootdir}/img_array_gridandfiller_imagenorm_{subset_id}_test.npy', imgs[test_idx])

In [ ]:
def make_test_data(geometry, pars, imgs):
    print(f"Beginning setting up training data for {geometry}")
    train_idx, test_idx = train_test_split(np.arange(len(pars)), 
                                           test_size=.2, random_state=99)
    
    
    parameters_to_fit = ['star.radius', 'star.temperature', 'envelope.rho_0',
     'envelope.power', 'ambient.density', 'ambient.temperature', 'scattering',
    ]
    if imgs.shape[2] > 1:
        # if the images are 2d
        parameters_to_fit.append('inclination')
        
    pars_arrlike = np.array(pars[parameters_to_fit]).view(float).reshape(len(pars), len(parameters_to_fit))
    np.save(f'{rootdir}/param_arr_gridandfiller_{subset_id}_train_all.npy',
            pars_arrlike[train_idx]
           )
    np.save(f'{rootdir}/param_arr_gridandfiller_{subset_id}_test.npy',
            pars_arrlike[test_idx]
           )
    np.save(f'{rootdir}/img_array_gridandfiller_imagenorm_{subset_id}_train_all.npy', imgs[train_idx])
    np.save(f'{rootdir}/img_array_gridandfiller_imagenorm_{subset_id}_test.npy', imgs[test_idx])
    print(f"Done setting up training data for {geometry} with training shape {train_idx.shape} and testing shape {test_idx.shape}")

In [ ]:
make_test_data(geometry, pars, arr)

## Try a geometry that requires 2D

In [87]:
geometry = 'spubsmi'

In [88]:
gridpath = f'/blue/adamginsburg/richardson.t/research/flux/grids-1.1/{geometry}/output/'

In [89]:
cd /blue/adamginsburg/richardson.t/research/flux/

/blue/adamginsburg/richardson.t/research/flux


In [90]:
pars = Table.read(f'robitaille_models-1.2/{geometry}/augmented_parameters.fits')

In [91]:
npars = len(pars)
npars

360000

In [92]:
mn = pars['MODEL_NAME'][0]
tem = model.ModelOutput(f'{gridpath}/{mn[:2]}/{mn[:-3]}.rtout').get_quantities()['temperature'][0].array
tem.shape

(1, 300, 400)

In [93]:
filesize = (npars * np.product(qtys['temperature'].shape) * 4 * u.byte).to(u.GB)
filesize

<Quantity 172.8 Gbyte>

In [94]:
npars = 10000

In [95]:
filesize = (npars * np.product(qtys['temperature'].shape) * 4 * u.byte).to(u.GB)
filesize

<Quantity 4.8 Gbyte>

In [97]:
arr = np.memmap(f'{rootdir}/{geometry}_temperature_grid_for_ML.npy',
                shape=(npars, *tem.shape),
                mode='w+')

In [ ]:
from astropy import log
log.setLevel(0)

for ii, mn in tqdm(enumerate(pars['MODEL_NAME'][:npars])):
    arr[ii, :, :, :] = (model
                        .ModelOutput(f'{gridpath}/{mn[:2].lower()}/{mn[:-3]}.rtout')
                        .get_quantities()['temperature'][0].array
                       )
    
log.setLevel('INFO')

0it [00:00, ?it/s]

In [ ]:
make_test_data(geometry, pars, arr)

# Following PPDAE Data Exploration notebook


# Do the training?

but don't use the script, instead use pieces grabbed from it - will reconstruct script later...

the script wasn't packaged right so it can't run easily...

In [39]:
%run $rootdir/PPDAE/ppdae/scripts/ae_main.py --help

usage: ae_main.py [-h] [--dry-run] [--machine MACHINE] [--data DATA]
                  [--img-norm IMG_NORM] [--par-norm PAR_NORM]
                  [--subset SUBSET] [--optim OPTIM] [--lr LR]
                  [--lr-sch LR_SCH] [--loss-fx LOSS_FX]
                  [--batch-size BATCH_SIZE] [--num-epochs NUM_EPOCHS]
                  [--early-stop] [--cond COND] [--feed-phy FEED_PHY]
                  [--latent-dim LATENT_DIM] [--dropout DROPOUT]
                  [--kernel-size KERNEL_SIZE] [--conv-blocks CONV_BLOCKS]
                  [--model-name MODEL_NAME] [--comment COMMENT]

AutoEncoder

optional arguments:
  -h, --help            show this help message and exit
  --dry-run             Load data and initialize models [False]
  --machine MACHINE     were to is running (local, colab, exalearn, hpg)
  --data DATA           data used for training ([Robitaille], MNIST, PPD)
  --img-norm IMG_NORM   type of normalization for images (global, [image])
  --par-norm PAR_NORM   physical p

In [42]:
cd $rootdir

/orange/adamginsburg/robitaille_models/ML_PPDAE


In [44]:
import wandb
wandb.login(relogin=True)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/adamginsburg/.netrc


True

In [48]:
wandb.init(project='ppdae', entity='ufsf')

wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [53]:
import importlib as imp
import ppdae.dataset_large
imp.reload(ppdae.dataset_large)

<module 'ppdae.dataset_large' from '/orange/adamginsburg/robitaille_models/ML_PPDAE/PPDAE/ppdae/dataset_large.py'>

In [54]:
%run $rootdir/PPDAE/ppdae/scripts/ae_main.py --latent-dim 16 --batch-size 128 --machine hpg --data Robitaille --dry-run

Running in:  hpg 

        dry_run	: True
        machine	: hpg
           data	: Robitaille
       img_norm	: image
       par_norm	: T
         subset	: 25052021
          optim	: Adam
             lr	: 0.0001
         lr_sch	: None
        loss_fx	: bce
     batch_size	: 128
     num_epochs	: 100
     early_stop	: False
           cond	: T
       feed_phy	: F
     latent_dim	: 16
        dropout	: 0.2
    kernel_size	: 3
    conv_blocks	: 5
     model_name	: ConvLinTrans_AE
         entity	: ufsf
        comment	: 
Using device cpu
Dataset size:  10000


wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Physic dimension:  7
Summary:
ConvLinTrans_AE(
  (enc_conv_blocks): Sequential(
    (conv2d_11): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn_11): BatchNorm2d(2, eps=1e-05, momentum=0.005, affine=True, track_running_stats=True)
    (relu_11): ReLU()
    (conv2d_12): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn_12): BatchNorm2d(2, eps=1e-05, momentum=0.005, affine=True, track_running_stats=True)
    (relu_12): ReLU()
    (maxpool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_21): Conv2d(2, 4, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn_21): BatchNorm2d(4, eps=1e-05, momentum=0.005, affine=True, track_running_stats=True)
    (relu_21): ReLU()
    (conv2d_22): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn_22): BatchNorm2d(4, eps=1e-05, momentum=0.005, affine=True, track_running_stats=True)
    (relu_22): ReLU()
    (maxpool_2): MaxPool2d(kernel_size=2, stride=2, padd


CondaError: Expected exactly one `egg-info` directory in '/blue/adamginsburg/adamginsburg/repos/image_tools', via egg-link 'lib/python3.9/site-packages/UNKNOWN.egg-link'. Instead found: ('image_tools.egg-info', 'UNKNOWN.egg-info').  These are often left over from legacy operations that did not clean up correctly.  Please remove all but one of these.



## This example fails because our data are only 1D

In [56]:
%run $rootdir/PPDAE/ppdae/scripts/ae_main.py --latent-dim 16 --batch-size 128 --machine hpg --data Robitaille

Running in:  hpg 

        dry_run	: False
        machine	: hpg
           data	: Robitaille
       img_norm	: image
       par_norm	: T
         subset	: 25052021
          optim	: Adam
             lr	: 0.0001
         lr_sch	: None
        loss_fx	: bce
     batch_size	: 128
     num_epochs	: 100
     early_stop	: False
           cond	: T
       feed_phy	: F
     latent_dim	: 16
        dropout	: 0.2
    kernel_size	: 3
    conv_blocks	: 5
     model_name	: ConvLinTrans_AE
         entity	: ufsf
        comment	: 
Using device cpu
Dataset size:  10000


wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Physic dimension:  7
Summary:
ConvLinTrans_AE(
  (enc_conv_blocks): Sequential(
    (conv2d_11): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn_11): BatchNorm2d(2, eps=1e-05, momentum=0.005, affine=True, track_running_stats=True)
    (relu_11): ReLU()
    (conv2d_12): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn_12): BatchNorm2d(2, eps=1e-05, momentum=0.005, affine=True, track_running_stats=True)
    (relu_12): ReLU()
    (maxpool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_21): Conv2d(2, 4, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn_21): BatchNorm2d(4, eps=1e-05, momentum=0.005, affine=True, track_running_stats=True)
    (relu_21): ReLU()
    (conv2d_22): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn_22): BatchNorm2d(4, eps=1e-05, momentum=0.005, affine=True, track_running_stats=True)
    (relu_22): ReLU()
    (maxpool_2): MaxPool2d(kernel_size=2, stride=2, padd

RuntimeError: Calculated padded input size per channel: (1 x 400). Kernel size: (3 x 3). Kernel size can't be greater than actual input size


CondaError: Expected exactly one `egg-info` directory in '/blue/adamginsburg/adamginsburg/repos/crowdsource', via egg-link 'lib/python3.9/site-packages/crowdsource.egg-link'. Instead found: ('crowdsource.egg-info', 'crowdsourcephoto.egg-info').  These are often left over from legacy operations that did not clean up correctly.  Please remove all but one of these.

